# Mnist Classification with Convolutional Neural Networks

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms

#### Get the dataset

First we get (download) the dataset and feed it to the dataset loaders

In [2]:
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor())

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

### Define the model

Here we use convolutional and max-pooling layers. The output doesn't have softmax because it is already computed inside the CrossEntropyLoss.

**TODO: How can we avoid computing by hand the input size of the fc layer?**

In [3]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)  ### 320 mjust be computed by the programmer!

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(self.conv1(x)).relu()
        x = self.mp(self.conv2(x)).relu()
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return x


model = ConvNet()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = torch.nn.CrossEntropyLoss()

#### Train and test the model

Nothing new for the training phase.
Since some layers behave differently during train and test phase (e.g. Dropout, BatchNorm), we need to use model.train and model.eval to set the phase

In [4]:
def train(epoch):
    
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
            
            
def test():
    
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
for epoch in range(1, 10):
    train(epoch)
    test()

/home/ralbertazzi/.local/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0029, Accuracy: 9449/10000 (94%)


Test set: Average loss: 0.0021, Accuracy: 9617/10000 (96%)


Test set: Average loss: 0.0016, Accuracy: 9678/10000 (96%)


Test set: Average loss: 0.0012, Accuracy: 9771/10000 (97%)


Test set: Average loss: 0.0010, Accuracy: 9792/10000 (97%)


Test set: Average loss: 0.0011, Accuracy: 9786/10000 (97%)


Test set: Average loss: 0.0009, Accuracy: 9814/10000 (98%)


Test set: Average loss: 0.0008, Accuracy: 9842/10000 (98%)


Test set: Average loss: 0.0009, Accuracy: 9828/10000 (98%)

